In [53]:
import pandas as pd
import os

In [54]:
files = list(filter(lambda s: s.startswith('data'), os.listdir()))
files

['data-8180-2022-05-17.json',
 'data-8174-2022-05-11.json',
 'data-49169-2022-05-12.json']

In [55]:
def read_decode(fn):
    with open(fn, encoding='cp1251') as f:
        return f.read()

In [56]:
df = pd.concat([pd.read_json(read_decode(file)) for file in files])

In [57]:
#!pip install geojson
import json
import geojson

In [58]:
df['point'] = df['geoData'].map(lambda p: geojson.Point(tuple(p['coordinates'])))

In [59]:
df['features'] = df.apply(lambda d: geojson.Feature(geometry=d['point'], properties={'District': d['District']}), axis=1)

In [60]:
fc = geojson.FeatureCollection(list(df['features']))
with open('features.geojson', 'w') as output:
    output.write(geojson.dumps(fc, ensure_ascii=False, indent=2))

In [61]:
districts = sorted(df['District'].unique())
with open('districts.json', 'w') as output:
    output.write(json.dumps(districts, ensure_ascii=False, indent=2))